# Import Gender Data to Train Model

In [1]:
import random
from nltk.corpus import names

# Read the names from the files.
# Label each name with the corresponding gender.
male_names = [(name, 'male') for name in names.words('male.txt')]
female_names = [(name, 'female') for name in names.words('female.txt')]

# Combine the lists.
labeled_names = male_names + female_names

# Take out leading and ending space characters from names
clean_names = [(name.strip(), gender) for (name, gender) in labeled_names]

# Shuffle the list, set seed for reproducibility
random.seed(74)
random.shuffle(clean_names)

# Create Features for Model

In [2]:
import pandas as pd
import numpy as np

# Create features for the model
# Not many possible, will settle for last letter and length of name
featuresets = [(n[-1],len(n),gender) for (n, gender) in clean_names]

labels = ['last_letter','name_length', 'gender']
df = pd.DataFrame.from_records(featuresets, columns=labels)
# Covert gender to binary variable
df['female'] = np.where(df['gender'] == 'female', 1, 0)
df.drop(['gender'], axis=1, inplace=True)

len(df)
# dataframe has 7,944 names with two features each

df.name_length.unique()
# All names have at least 2 characters

df.last_letter.unique()
# All entries end with a letter
# Note that there are no names ending in 'q', so no dummy variable will be prepared for this

array(['o', 'e', 'a', 'n', 'm', 's', 'r', 't', 'y', 'd', 'i', 'k', 'h',
       'p', 'l', 'j', 'c', 'w', 'b', 'g', 'f', 'v', 'u', 'x', 'z'], dtype=object)

In [3]:
pd.crosstab(df['last_letter'],df['female'], dropna=False, margins=True)
# Among frequent (n>100) names:
# Names ending with 'a','e','i' tend to be female
# While names ending with 'd','o','r','s','t' tend be male
pd.crosstab(df['name_length'],df['female'], dropna=False, margins=True)
# Longer names (>4 letters) tend to be female (~64% of longer names in the dataset)

female,0,1,All
name_length,,,
2,11,8,19
3,147,125,272
4,390,537,927
5,660,1217,1877
6,791,1258,2049
7,495,952,1447
8,296,550,846
9,106,245,351
10,33,83,116


# Train and Test Models

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

pd.options.mode.chained_assignment = None  # default='warn'

alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# Seperate out predictors (X or features) and response variable (y)
X_df = df[['last_letter','name_length']]
# Add 'q' as a category so that a dummy variable will be created for the letter 'q'
X_df['last_letter'] = df['last_letter'].astype('category', categories= alphabet)
y_df = df[['female']]
y_val = y_df['female'].values
# One-hot encode the last_letter categorical variable
X_dummies = pd.get_dummies(X_df, columns=['last_letter'])

# Create train/test split to train and optimize models
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_df, random_state=74)
y_train = y_train['female'].values

# Scale X features for SVM model
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dummies_scaled = scaler.fit_transform(X_dummies)

# Baseline Model: Naive Bayes Gaussian
# Use cross-fold validation to evaluate each model's performance
# Note that names dataset contains ~63% female names
gnb_clf = GaussianNB()
gnb_score = cross_val_score(gnb_clf, X_dummies, y_val, cv=3, scoring = 'accuracy').mean()
# 73.40%

# Non-linear model: SVM

# Use GridSearchCV to optimize the model using the training set

#svm_clf = SVC(kernel='rbf')
#grid_values = {'gamma': [0.01, 0.1, 1, 10],
                  #'C': [0.1,1,10,100]}
#grid_svmclf_acc = GridSearchCV(svm_clf, param_grid = grid_values)
#grid_svmclf_acc.fit(X_train_scaled, y_train)
#grid_svmclf_acc.best_params_
# C = 10, gamma = 10

svm_clf_opt = SVC(kernel='rbf', C=10, gamma=10)
svm_score = cross_val_score(svm_clf_opt, X_dummies_scaled, y_val, cv=3, scoring = 'accuracy').mean()
# 76.66%

# Decision-tree model: Gradient Boosting Decision Trees

#gbdt_clf = GradientBoostingClassifier()
#grid_values = {'learning_rate': [0.01, 0.1, 0.3, 0.5],
                  #'n_estimators': [50,100,200,300]}
#grid_gbdtclf_acc = GridSearchCV(gbdt_clf, param_grid = grid_values)
#grid_gbdtclf_acc.fit(X_train, y_train)
#grid_gbdtclf_acc.best_params_
# learning rate = 0.1, n estimators = 300

gbdt_clf_opt = GradientBoostingClassifier(learning_rate=0.1, n_estimators=300)
gbdt_score = cross_val_score(gbdt_clf_opt, X_dummies, y_val, cv=3, scoring = 'accuracy').mean()
# 76.71, very slightly better than the SVM model

# Final model chosen: GBDT

# Import Sunshine List Names Data

In [5]:
import glob
import re

file_path = '/Users/arielaguilargonzalez/Library/Lima/Python/Sunshine List Gender Prediction Model/Data'
allFiles = glob.glob(file_path + "/*.csv")

# Import dataframes into a list
allyears = []
for file in allFiles:
    df = pd.read_csv(file,index_col=None, header=0)
    # Set the column names for each year
    df.columns = ['sector','last_name', 'first_name', 'salary_paid', 'taxable_benefits', 'employer', 'job_title', 'calendar_year']
    # Calendar_year variable is messy, set equal to the first observation, verified as clean
    df['calendar_year'] = df['calendar_year'][0]
    allyears.append(df)
    
# Concatenate all dataframes into one big dataframe
sl_allyears = pd.concat(allyears, ignore_index=True)
len(sl_allyears)
# 538 478 observations
sl_allyears['calendar_year'].unique()
# Have right years displayed, 2012-2016

# Check for duplicates
len(sl_allyears[sl_allyears.duplicated()])
# 0, no duplicates

# How many employees were in each year?
sl_allyears.groupby('calendar_year')['last_name'].count()
# Number has grown every year from 88 542 in 2012 to 124 445 in 2016

calendar_year
2012     88542
2013     97916
2014    111655
2015    115920
2016    124445
Name: last_name, dtype: int64

# Clean Up First Names

In [6]:
# Any null first_names?
sl_allyears['first_name'].isnull().sum()
# 1, remove
sl_allyears.dropna(subset=['first_name'], inplace=True)
len(sl_allyears)
# 538 477 observations

sl_allyears['first_name_clean'] = sl_allyears['first_name'].str.strip()
# Remove leading and trailing whitespace
sl_allyears['first_name_clean'] = sl_allyears['first_name_clean'].str.lower()
# Lowercase all letters
sl_allyears['first_name_clean'] = sl_allyears['first_name_clean'].str.encode('ascii', 'ignore')
# Convert strings to ascii to convert accented characters to plain letters
sl_allyears['first_name_clean'] = sl_allyears['first_name_clean'].str.decode('UTF-8')
# Convert ascii characters back to UTF
sl_allyears['first_name_regex'] = sl_allyears['first_name_clean'].str.extract(r'([a-z]{2,})')
# Regex only extracts whole names (two characters or more) so entries like bruce j. become bruce
# Extract takes the first match, so a name like marie lise would be extracted as marie

# Any non-matches?
sl_allyears['first_name_regex'].isnull().sum()
# 17 non-matches, look at data
sl_allyears[sl_allyears['first_name_regex'].isnull()]
# These names consist of only initials e.g A.J. , drop
sl_allyears.dropna(subset=['first_name_regex'], inplace=True)
len(sl_allyears)
# 538 460 observations

/Users/arielaguilargonzalez/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


538460

# Create Gender Predictions For Names

In [7]:
distinct_names = sl_allyears['first_name_regex'].drop_duplicates()
len(distinct_names)
# 16 124 distinct names

16124

In [8]:
slnames_features = [(w[-1:],len(w)) for w in distinct_names]

labels = ['last_letter','name_length']
slfeatures_df = pd.DataFrame.from_records(slnames_features, columns=labels)
slfeatures_df['last_letter'] = slfeatures_df['last_letter'].astype('category', categories= alphabet)
sl_dummies = pd.get_dummies(slfeatures_df, columns=['last_letter'])

# Fit the final classifier model with optimal paramters and full training set
gender_clf = GradientBoostingClassifier(learning_rate=0.1, n_estimators=300).fit(X_dummies, y_val)

predictions = gender_clf.predict_proba(sl_dummies)
# First column is probability that class = 0, male
# Second column is prob that class = 1, female

# Merge predictions with distinct names
fem_pred = pd.DataFrame(predictions[:,1])
d_names = pd.DataFrame(distinct_names).reset_index()

dnames_pred = pd.merge(d_names, fem_pred, how='left', left_index=True, right_index=True)
dnames_pred.drop(['index'], axis=1, inplace=True)
dnames_pred.columns = ['distinct_name', 'fem_prob']
dnames_pred.head()

,distinct_name,fem_prob
0,sadiq,0.421819
1,jeffery,0.525883
2,abdul,0.514695
3,gail,0.456675
4,ian,0.358400


# Merge Gender Predictions to Data

In [11]:
sl_gender = pd.merge(sl_allyears, dnames_pred, how='left', left_on=['first_name_regex'], right_on=['distinct_name'])
sl_gender.drop(['first_name_clean', 'first_name_regex'], axis=1, inplace=True)
# Create gender variable
sl_gender['gender'] = np.where(sl_gender['fem_prob']> 0.50, 'female', 'male')
sl_gender.head()

,sector,last_name,first_name,salary_paid,taxable_benefits,employer,job_title,calendar_year,distinct_name,fem_prob,gender
0,Colleges,ABBAS,SADIQ,102186,119.88,Algonquin College,Professor,2012,sadiq,0.421819,male
1,Colleges,AGATE,JEFFERY MICHAEL,104657,335.15,Algonquin College,"Manager, Counselling and Testing Services",2012,jeffery,0.525883,female
2,Colleges,AL-AZZAWI,ABDUL,103133,185.88,Algonquin College,Professor,2012,abdul,0.514695,female
3,Colleges,ALLAN,GAIL,104893,185.88,Algonquin College,Professor,2012,gail,0.456675,male
4,Colleges,ALLEN,IAN D.,104141,119.88,Algonquin College,Professor,2012,ian,0.358400,male


In [12]:
# What percent of predictions are confident? i.e fall outside 45-55% band
(len(sl_gender[sl_gender['fem_prob'] > 0.55]) + len(sl_gender[sl_gender['fem_prob'] < 0.45]))/len(sl_gender)
# ~86%

# Outside 40-60% band?
(len(sl_gender[sl_gender['fem_prob'] > 0.60]) + len(sl_gender[sl_gender['fem_prob'] < 0.40]))/len(sl_gender)
# ~81%

0.8061991605690302

# Clean Salary_Paid Variable

In [13]:
import re

# Check data types within salary_paid
salary_dtypes = set([type(x) for x in sl_gender['salary_paid']])
salary_dtypes
# Have string and float types

clean_salary = []

for salary in sl_gender['salary_paid']:
    if type(salary) == str:
        salary_regex = float(re.sub(r'\$|\,','', salary))
        clean_salary.append(salary_regex)
    else:
        clean_salary.append(salary)

len(clean_salary)
# 538 640 salaries

salary_float = pd.DataFrame(clean_salary)
salary_float.columns = ['salary_num']
salary_float.describe()
# Minimum is $100k, max is $1.72 million

# Merge clean salary series onto sl_gender dataframe
sl_salclean = pd.merge(sl_gender, salary_float, how="left", left_index=True, right_index=True)
sl_salclean.head()

,sector,last_name,first_name,salary_paid,taxable_benefits,employer,job_title,calendar_year,distinct_name,fem_prob,gender,salary_num
0,Colleges,ABBAS,SADIQ,102186,119.88,Algonquin College,Professor,2012,sadiq,0.421819,male,102185.98
1,Colleges,AGATE,JEFFERY MICHAEL,104657,335.15,Algonquin College,"Manager, Counselling and Testing Services",2012,jeffery,0.525883,female,104656.60
2,Colleges,AL-AZZAWI,ABDUL,103133,185.88,Algonquin College,Professor,2012,abdul,0.514695,female,103133.47
3,Colleges,ALLAN,GAIL,104893,185.88,Algonquin College,Professor,2012,gail,0.456675,male,104893.10
4,Colleges,ALLEN,IAN D.,104141,119.88,Algonquin College,Professor,2012,ian,0.358400,male,104140.79


# Clean Employer Variable

In [14]:
sl_salclean['employer_strip'] = sl_salclean['employer'].str.strip()
# Remove leading and trailing whitespace
sl_salclean['employer_strip'] = sl_salclean['employer_strip'].str.lower()
# lower case all letters
sl_salclean['employer_strip'] = sl_salclean['employer_strip'].str.replace(r'\&', 'and')
# replace '&' with 'and'

In [15]:
# Create Sentence Comparator function
# Will use this function to identify potential duplicate employers

import nltk.corpus
import nltk.stem.snowball
import string
from nltk.tokenize import TreebankWordTokenizer

# Get default English stopwords and extend with punctuation
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(string.punctuation)
stopwords.append('')

# Create tokenizer and stemmer
tokenizer = TreebankWordTokenizer()

def sent_similar_score(a, b):
    """Check if a and b are matches."""
    tokens_a = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(a) \
                    if token.lower().strip(string.punctuation) not in stopwords]
    tokens_b = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(b) \
                    if token.lower().strip(string.punctuation) not in stopwords]

    # Calculate Jaccard similarity
    ratio = len(set(tokens_a).intersection(tokens_b)) / float(len(set(tokens_a).union(tokens_b)))
    return (ratio)

In [16]:
employers = sl_salclean['employer_strip'].unique().tolist()
# Unique employer names
employers.sort()
# Sort alphabetically

# Create lookup series to match potential duplicate names to de-dup names

employer_join = []

for i in range(len(employers)):
    if i == 0:
        employer_join.append(employers[i])
        # Start the lookup list with the first raw employer name, verified as unique
    else:
        score = sent_similar_score(employers[i], employer_join[i-1])
        # Compare the similarity between the raw employer name and the preceeding name on the lookup list
        if score < 0.9:
        # Set conservative threshold, employer names have to be almost exact for them to be replaced
               employer_join.append(employers[i])
        else:
               employer_join.append(employer_join[i-1])

len(set(employer_join))
# 2 800 employer names from original number of ~3 290 names

2800

In [17]:
# Merge the lookup clean employer name series to the raw employer name
employers_df = pd.DataFrame(employers)
employerjoin_df = pd.DataFrame(employer_join)

employer_lookup = pd.concat([employers_df, employerjoin_df], axis=1)
employer_lookup.columns = ['raw_employer', 'clean_employer']

In [18]:
# Merge clean employer_lookup onto sl_salclean dataframe
sl_empclean = pd.merge(sl_salclean, employer_lookup, how="left", left_on=['employer_strip'], right_on=['raw_employer'])
sl_empclean.drop(['employer_strip', 'raw_employer'], axis=1, inplace=True)
sl_empclean.head()

,sector,last_name,first_name,salary_paid,taxable_benefits,employer,job_title,calendar_year,distinct_name,fem_prob,gender,salary_num,clean_employer
0,Colleges,ABBAS,SADIQ,102186,119.88,Algonquin College,Professor,2012,sadiq,0.421819,male,102185.98,algonquin college
1,Colleges,AGATE,JEFFERY MICHAEL,104657,335.15,Algonquin College,"Manager, Counselling and Testing Services",2012,jeffery,0.525883,female,104656.60,algonquin college
2,Colleges,AL-AZZAWI,ABDUL,103133,185.88,Algonquin College,Professor,2012,abdul,0.514695,female,103133.47,algonquin college
3,Colleges,ALLAN,GAIL,104893,185.88,Algonquin College,Professor,2012,gail,0.456675,male,104893.10,algonquin college
4,Colleges,ALLEN,IAN D.,104141,119.88,Algonquin College,Professor,2012,ian,0.358400,male,104140.79,algonquin college


# Clean Sector Variable

In [19]:
sl_empclean['sector'].unique()

array(['Colleges', 'Crown Agencies', 'Government of Ontario - Judiciary',
       'Government of Ontario - Legislative Assembly and Offices',
       'Government of Ontario - Ministries',
       'Hospitals and Boards of Public Health',
       'Hydro One and Ontario Power Generation',
       'Municipalities and Services', 'Other Public Sector Employers',
       'School Boards', 'Seconded (Children & Youth Services)*',
       'Seconded (Citizenship & Immigration)*',
       'Seconded (Comm. Safety and Corr. Services)*',
       'Seconded (Education)*', 'Seconded (Health & Long–Term Care)*',
       'Seconded (Training, Colleges & Universities)*', 'Universities',
       'Seconded (Tourism, Culture & Sport)*',
       'Seconded (Transportation)*',
       'Seconded (Training, Colleges and Universities)*',
       'Seconded (Children and Youth Services)*',
       'Seconded (Health and Long-Term Care)*',
       'Seconded (Community and Social Services)*',
       'Government of Ontario - Legislative 

In [24]:
sl_salclean['sector_clean'] = sl_salclean['sector'].str.strip()
# Remove leading and trailing whitespace
sl_salclean['sector_clean'] = sl_salclean['sector_clean'].str.lower()
# lower case all letters
sl_salclean['sector_clean'] = sl_salclean['sector_clean'].str.replace(r'\&', 'and')
# replace '&' with 'and'
sl_salclean['sector_clean'] = sl_salclean['sector_clean'].str.replace(r'seconded .*', 'government of ontario - ministries')
# seconded employees work for a public organization but have been borrowed by a government ministry
sl_salclean['sector_clean'].unique()
# merge seperate opg sector into hydro one/opg
sl_salclean['sector_clean'] = np.where(sl_salclean['sector_clean'] == 'ontario power generation', 'hydro one and ontario power generation', sl_salclean['sector_clean'])
sl_salclean['sector_clean'].unique()

array(['colleges', 'crown agencies', 'government of ontario - judiciary',
       'government of ontario - legislative assembly and offices',
       'government of ontario - ministries',
       'hospitals and boards of public health',
       'hydro one and ontario power generation',
       'municipalities and services', 'other public sector employers',
       'school boards', 'universities'], dtype=object)

# Export Final Dataset to CSV

In [29]:
sl_salclean.head()
sl_clean = sl_salclean.loc[:,['distinct_name', 'sector_clean','employer_strip', 'salary_num', 'fem_prob', 'gender','calendar_year']]
sl_clean.columns = ['first_name', 'sector', 'employer', 'salary', 'fem_prob', 'gender','year']
sl_clean.head()

,first_name,sector,employer,salary,fem_prob,gender,year
0,sadiq,colleges,algonquin college,102185.98,0.421819,male,2012
1,jeffery,colleges,algonquin college,104656.60,0.525883,female,2012
2,abdul,colleges,algonquin college,103133.47,0.514695,female,2012
3,gail,colleges,algonquin college,104893.10,0.456675,male,2012
4,ian,colleges,algonquin college,104140.79,0.358400,male,2012


In [30]:
# Limit dataset to confident predictions (outside 45-55% range)
sl_conf = sl_clean.loc[(sl_clean['fem_prob'] > 0.55) | (sl_clean['fem_prob'] < 0.45)]

In [31]:
sl_conf.to_csv('/Users/arielaguilargonzalez/Library/Lima/Python/Sunshine List Gender Prediction Model/Output/Sunshine_List_Conf.csv')